##### 【 의사결정나무 알고리즘 - DT + CV】
- iris(붓꽃) 품종 분류 모델
- 학습종류 : 지도학습 - 분류
- 학습방법 : DT기반 분류  

[1] 모듈 로딩 및 데이터 준비<hr>

In [19]:
## ==================================================
## [1-1] 모듈 로딩
## ==================================================
import pandas as pd 
import numpy as np

## ML학습 관련
from sklearn.tree import DecisionTreeClassifier

## ML 데이터셋 및 전처리 관련
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, LabelEncoder

## ML 성능지표 관련
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score
from sklearn.metrics import classification_report 

## 시각화 관련
import matplotlib.pyplot as plt
import graphviz

In [20]:
## ==================================================
## [1-2] 데이터 준비 및 확인
## ==================================================
## 데이터
DATA_FILE = '../Data/iris.csv'

## 데이터 로딩
irisDF = pd.read_csv(DATA_FILE)

## 데이터 기본정보 확인
display( irisDF.head(2) )
irisDF.info()


,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal.length  150 non-null    float64
 1   sepal.width   150 non-null    float64
 2   petal.length  150 non-null    float64
 3   petal.width   150 non-null    float64
 4   variety       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


[2] 학습 준비<hr>

In [21]:
## ==================================================
## [2-1] 피쳐/타겟 분리
## ==================================================
featureDF = irisDF[irisDF.columns[:-1]]
targetSR  = irisDF[irisDF.columns[-1]]

print(f'featureDF:{featureDF.shape},  targetSR:{targetSR.shape}')

featureDF:(150, 4),  targetSR:(150,)


In [22]:
## ==================================================
## [2-2] 학습용/테스트용 분리
## ==================================================
x_train, x_test, y_train, y_test = train_test_split(featureDF,
                                                    targetSR,
                                                    test_size=0.2,
                                                    random_state=42,
                                                    stratify=targetSR)

print(f'[TRAIN] x_train:{x_train.shape},  y_train:{y_train.shape}')
print(f'[TEST] x_test:{x_test.shape},  y_test:{y_test.shape}')

[TRAIN] x_train:(120, 4),  y_train:(120,)
[TEST] x_test:(30, 4),  y_test:(30,)


[3] 학습 진행 <hr>

In [ ]:
## ===================================================================
## 교차검증을 통한 일반화 성능 체크 + 최적의 하이퍼파라미터 찾기
## ===================================================================
from sklearn.model_selection import StratifiedKFold

## 분류용 교차검증 + 재현성 
skFold = StratifiedKFold(random_state=42, shuffle=True)

## DT의 하이퍼파라미터 중 max_depth에 따른 성능 저장 
CV_RESULTS = {'DEPTH':[], 'TRAIN':[], 'VALID':[]}

for depth in range(1, 21):
    print(f'\n[{depth}] CV:{skFold.n_splits} 개-----------------------')

    CV_TRAIN, CV_VALID = 0, 0
    for train_indices, valid_indices in skFold.split(x_train, y_train):
        ##-> 학습용/검증용 데이터셋 분리
        x_trainDF = x_train.iloc[train_indices]
        y_trainSR = y_train.iloc[train_indices]

        x_validDF = x_train.iloc[valid_indices]
        y_validSR = y_train.iloc[valid_indices]

    
        ##-> 데이터셋 전처리
        lb_encoder = LabelEncoder()
        en_y_train = lb_encoder.fit_transform(y_trainSR)
        en_y_valid = lb_encoder.transform(y_validSR)

        rb_scaler = RobustScaler()
        rb_x_train = rb_scaler.fit_transform(x_trainDF)
        rb_x_valid = rb_scaler.transform(x_validDF)

        ##-> 학습 진행
        treeModel = DecisionTreeClassifier(random_state=42, max_depth=depth)
        treeModel.fit(rb_x_train, en_y_train)

        ##-> 평가
        train_score  = treeModel.score(rb_x_train, en_y_train)
        valid_score  = treeModel.score(rb_x_valid, en_y_valid)

        CV_TRAIN += train_score
        CV_VALID += valid_score
    
    ## max_depth마다 모델 일반화 성능 저장 
    CV_RESULTS['DEPTH'].append(depth)
    CV_RESULTS['TRAIN'].append(CV_TRAIN/skFold.n_splits)
    CV_RESULTS['VALID'].append(CV_VALID/skFold.n_splits)
    print(CV_RESULTS)

## 전체 결과 출력 
resultDF = pd.DataFrame(CV_RESULTS)
display(resultDF)


[1] CV:5 개-----------------------
{'DEPTH': [1], 'TRAIN': [0.6666666666666666], 'VALID': [0.6666666666666666]}

[2] CV:5 개-----------------------
{'DEPTH': [1, 2], 'TRAIN': [0.6666666666666666, 0.9666666666666668], 'VALID': [0.6666666666666666, 0.9416666666666667]}

[3] CV:5 개-----------------------
{'DEPTH': [1, 2, 3], 'TRAIN': [0.6666666666666666, 0.9666666666666668, 0.9791666666666666], 'VALID': [0.6666666666666666, 0.9416666666666667, 0.9583333333333334]}

[4] CV:5 개-----------------------
{'DEPTH': [1, 2, 3, 4], 'TRAIN': [0.6666666666666666, 0.9666666666666668, 0.9791666666666666, 0.99375], 'VALID': [0.6666666666666666, 0.9416666666666667, 0.9583333333333334, 0.9416666666666667]}

[5] CV:5 개-----------------------
{'DEPTH': [1, 2, 3, 4, 5], 'TRAIN': [0.6666666666666666, 0.9666666666666668, 0.9791666666666666, 0.99375, 0.9979166666666668], 'VALID': [0.6666666666666666, 0.9416666666666667, 0.9583333333333334, 0.9416666666666667, 0.95]}

[6] CV:5 개-----------------------
{'DEPTH': [

,DEPTH,TRAIN,VALID
0,1,0.666667,0.666667
1,2,0.966667,0.941667
2,3,0.979167,0.958333
3,4,0.993750,0.941667
4,5,0.997917,0.950000
5,6,1.000000,0.950000
6,7,1.000000,0.950000
7,8,1.000000,0.950000
8,9,1.000000,0.950000
9,10,1.000000,0.950000
